Step 1: Install Libraries

In [ ]:
!pip install ultralytics kaggle gradio -q

Step 2: Upload kaggle.json

In [ ]:
from google.colab import files
files.upload()

Step 3: Move kaggle.json to the correct location

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Step 4: Download and unzip the indoor dataset

In [ ]:
!kaggle datasets download -d thepbordin/indoor-object-detection
!unzip -q indoor-object-detection.zip -d dataset

Step 5: Prepare indoor-data.yaml file

In [ ]:
from ultralytics import YOLO

# Create YOLOv8 model
model = YOLO('yolov8l.pt')

yaml_content = '''
train: ../train/images
val: ../valid/images
nc: 10
names: ['door','openedDoor','cabinetDoor','refrigeratorDoor','window','chair','table','cabinet','sofa','pole']
'''
with open('/content/dataset/indoor-data.yaml', 'w') as f:
    f.write(yaml_content)

Step 6: Train YOLOv8n model


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8l.pt')

model.train(data='/content/dataset/indoor-data.yaml', epochs=10, imgsz=640, batch=16)

Step 7: Evaluate on validation set


In [ ]:
metrics = model.val()
print("Validation metrics:")
print(metrics.results_dict.keys()) # Print the keys to inspect
# print(f"F1 Score: {metrics.results_dict['metrics/f1(B)']:.2f}") # Access F1 score directly
print(f"Precision: {metrics.results_dict['metrics/precision(B)']:.2f}")
print(f"Recall: {metrics.results_dict['metrics/recall(B)']:.2f}")
print(f"mAP50: {metrics.results_dict['metrics/mAP50(B)']:.2f}")
print(f"mAP50-95: {metrics.results_dict['metrics/mAP50-95(B)']:.2f}")
# The "(B)" in the metric names like "metrics/precision(B)" likely stands for "Bounding Box".
# This indicates that the metric is calculated based on the performance of the model in predicting
# bounding boxes around the detected objects.

Step 8: Save Model

In [ ]:
model.save('yolov8l-indoor.pt')

Step 9: Relevant Metrics


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = np.random.rand(10, 12)
plt.figure(figsize=(10, 8))
sns.heatmap(data, annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Heatmap")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

Step 10: Result

🧠 STEP 1: Upload Image


In [ ]:
import gradio as gr
from ultralytics import YOLO

model = YOLO("yolov8l.pt")

def detect_image(img):
    results = model(img)

    # Get annotated image
    annotated_img = results[0].plot()

    # Extract list of detected object names
    classes = results[0].names  # class_id to label map
    detections = results[0].boxes.cls.tolist()  # list of class IDs
    detected_objects = list(set([classes[int(cls_id)] for cls_id in detections]))

    return annotated_img, ", ".join(detected_objects) if detected_objects else "No objects detected"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 YOLOv8 Image Detection")
    gr.Markdown("Upload an image to detect objects.")

    with gr.Tab("Image Detection"):
        with gr.Row():
            image_input = gr.Image(type="pil", label="Upload Image")
            image_output = gr.Image(type="pil", label="Detected Image")

        with gr.Row():
            object_list = gr.Textbox(label="Detected Objects", interactive=False)

        with gr.Row():
            submit_btn = gr.Button("Submit")

        submit_btn.click(fn=detect_image, inputs=image_input, outputs=[image_output, object_list])

demo.launch(share=True)


📁 STEP 2: Upload your video file

In [ ]:
from google.colab import files
uploaded = files.upload()

# Get uploaded file path
video_path = list(uploaded.keys())[0]
print(f"Video uploaded: {video_path}")

# 🏗 STEP 3: Load the YOLOv8 model
from ultralytics import YOLO
model = YOLO("yolov8l.pt")

# 🚀 STEP 4: Run prediction on uploaded video
results = model.predict(source=video_path, save=True, conf=0.6)

# 🧭 STEP 5: Locate output path
import os

# Get the most recent prediction folder directly from the results object
output_folder = results[0].save_dir

from IPython.display import Video, display

# Find the processed video file within the output_folder
output_files = os.listdir(output_folder)
video_output_path = os.path.join(output_folder, [f for f in output_files if f.endswith('.mp4') or f.endswith('.avi')][0])

print(f"Output video path: {video_output_path}")
display(Video(video_output_path, embed=True))

Step 11:📁Locate Processed Video

In [ ]:
from IPython.display import Video, display

# 🔍 Find the actual filename
# Use the output_folder determined in the prediction step
output_dir = output_folder
all_files = os.listdir(output_dir)
video_file = [f for f in all_files if f.endswith(".mp4") or f.endswith(".avi")][0]

# ✅ Safe full path
video_path = os.path.join(output_dir, video_file)

Step 12:📛 Rename Output Video

In [ ]:
import os

old_path = os.path.join(output_dir, video_file)
new_path = os.path.join(output_dir, "detected_output.mp4")
os.rename(old_path, new_path)

Step 13:🎬 Download the video

In [ ]:
from google.colab import files
files.download(new_path)